In [2]:
import torch
from avalanche.training.supervised import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import TensorboardLogger
from torch.utils.tensorboard import SummaryWriter
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics,class_accuracy_metrics
from avalanche.benchmarks.classic import CORe50
from avalanche.logging import InteractiveLogger
from avalanche.training.supervised import Naive
from SimpleCNN import *
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from avalanche.benchmarks.utils import as_avalanche_dataset

In [3]:
from torchvision import transforms

# Trasformazione per ridurre la risoluzione delle immagini
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Riduci la risoluzione a 64x64 (o altra risoluzione desiderata)
    transforms.ToTensor()
])

In [4]:
# Caricamento del dataset CORe50
benchmark = CORe50(scenario="nc",run=0,object_lvl=False,train_transform=transform,eval_transform=transform)

# Unione di tutte le esperienze in un unico dataset
train_dataset = as_avalanche_dataset(benchmark.train_stream)
test_dataset = as_avalanche_dataset(benchmark.test_stream)

# Creazione di un'unica esperienza di addestramento con tutti i dati
train_experience = benchmark.train_stream[0]  # Consideriamo solo la prima esperienza per questo esempio
test_experience = benchmark.test_stream[0]


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified


In [5]:

# Specifica del percorso di salvataggio dei log per TensorBoard
log_dir = './tb_data/experiment_classic'

# Creazione del logger di TensorBoard per run1
tb_logger_run1 = TensorboardLogger(tb_log_dir=log_dir)

# Lista dei modelli
print("Classi Presenti : "+str(benchmark.n_classes))
model = SimpleCNN(num_classes=benchmark.n_classes)

print(f"Training model: {model.__class__.__name__}")


Classi Presenti : 10
Training model: SimpleCNN


In [6]:

# Plugin per la valutazione
eval_plugin = EvaluationPlugin(    
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),\
    class_accuracy_metrics(stream=True),\
    loggers=[tb_logger_run1])

# Definizione dell'ottimizzatore e della loss function
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Creazione della strategia di addestramento
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=10, eval_mb_size=100,
    evaluator=eval_plugin
)



c:\Users\tellg\Documents\Universita\VISIOPE\2023-2024\Progetto\ContinualLearning\.venv\lib\site-packages\avalanche\training\templates\base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [7]:
# Addestramento del modello utilizzando la strategia Naive di Avalanche
strategy.train(train_experience, eval_stream=[test_experience])

# Valutazione del modello
strategy.eval([test_experience])

{'Top1_Acc_MB/train_phase/train_stream/Task000': 1.0,
 'Loss_MB/train_phase/train_stream/Task000': 7.351227395702153e-07,
 'RunningTime_Epoch/train_phase/train_stream/Task000': 0.0006798477333334935,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0,
 'Loss_Epoch/train_phase/train_stream/Task000': 3.079392446770506e-05,
 'Time_Epoch/train_phase/train_stream/Task000': 210.37939890000007,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.23374544160811173,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 12.375832540499767,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.23374544160811173,
 'Loss_Stream/eval_phase/test_stream/Task000': 12.375832540499767,
 'StreamForgetting/eval_phase/test_stream': 0.0,
 'Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/0': 0.11691486997110469,
 'Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/1': 0.435720640569395,
 'Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/2': 0.3362222222222222,
 'Top1_ClassAcc_Stream/eval_phase/t